In [1]:
import spotipy
import csv
import os
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
from dotenv import load_dotenv

In [2]:
load_dotenv()

client_id = os.getenv('SPOTIPY_CLIENT_ID')
client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')
redirect_uri = os.getenv('SPOTIPY_REDIRECT_URI')

client_credentials_manager = SpotifyClientCredentials()
spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials())

In [3]:
genre_list = ['pop', 'rock', 'hip hop', 'rap', 'chillhop', 'metal', 'indie', 'house', 'country', 'edm']
genre_dict = dict.fromkeys(genre_list)

for genre in genre_dict:
    genre_dict[genre] = []

for genre in genre_list:
    search_result = spotify.search(q=genre, limit=50) #the result only consists of tracks

    genre_dict[genre].extend(search_result['tracks']['items'])

    for i in range(19):
        search_result = spotify.next(search_result['tracks'])
        genre_dict[genre].extend(search_result['tracks']['items'])

In [4]:
genre_dict['pop'][0].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [5]:
tracks_id = dict.fromkeys(genre_list)

for genre in tracks_id:
    tracks_id[genre] = []

for genre in genre_dict:
    for i in range(len(genre_dict[genre])):
        tracks_id[genre].append(genre_dict[genre][i]['id'])

for genre in tracks_id:
    tracks_id[genre] = list(dict.fromkeys(tracks_id[genre]))

In [6]:
track_features_list = dict.fromkeys(genre_list)

for genre in track_features_list:
    track_features_list[genre] = []

for genre in tracks_id:
    for i in range(len(tracks_id[genre])):
        track_features_list[genre].append(spotify.audio_features(tracks_id[genre][i]))

In [7]:
for genre in track_features_list:
    track_features_list[genre] = list(filter(lambda x: x != [None], track_features_list[genre]))

In [8]:
for genre in track_features_list: 
    for i in range(len(track_features_list[genre])):
        track_features_list[genre][i] = dict(list(filter(lambda x: x[0] == 'id' or type(x[1]) != str, list(track_features_list[genre][i][0].items()))))

In [9]:
for genre in track_features_list:
    for i in range(len(track_features_list[genre])):
        track_features_list[genre][i]['genre'] = genre

In [10]:
for genre in track_features_list:
    for i in range(len(track_features_list[genre])):
        track_features_list[genre][i] = {'id': track_features_list[genre][i].pop('id'), **track_features_list[genre][i]}

In [11]:
save_data = []
for genre in track_features_list:
    for i in range(len(track_features_list[genre])):
        save_data.append(track_features_list[genre][i])

In [12]:
with open('10_genres_audio_features.tsv', 'w') as file:
    writer = csv.DictWriter(file, fieldnames=save_data[0].keys(), delimiter='\t')
    writer.writeheader()
    for data in save_data:
        writer.writerow(data)